# Convnet Cheat Sheet

In this notebook, we do the following two tasks.

- Create a cheat sheet for convnets
- Implement the LeNet5 architecture

Let's consider the following general convnet architecture. We will state the expression for the number of learnable parameters and dimension of each layer.


<img src="https://cse.unl.edu/~hasan/Pics/CNN_Architecture_Cartoon_3D.png" width=800, height=600>

## Input Layer

Typically images are RGB and are large in size. Three types of processing are usually done on the input images.

- Downscale the Images

Large images are scaled to small sizes as they are expensive to process (need more RAM and parameters). Depending on the memory of CPU/GPU the scaling factor is determined. A preferred size for downscaling is 128 x 128. If space becomes an issue, we can downsample to 32 x 32.

- Augment the Images

To reduce overfitting, training data are augmented by slighty "distorting" the image via spatial translation, rotation, adding noise, etc.

- Standardize the Images

The Backpropagation algorithm for training convnets require us to standardize the images. One of the two techniques are used.

        -- Standardization: transform the images to have zero mean and unit variance.
        -- Scale (min-max scaling): transform the pixels to lie between 0 and 1.


## Conv Layer

Let's say that the input image has $n_k$ channels with height and width: $n_h \times n_w$. Then, a single filter will have $n_k$ channels. For a stride $s$, the dimension of a single filter is $f_h \times f_w \times n_k$.

The number of these 3D filters $f_k$ is usually a power of 2. For example, we use filters 2, 4, 8, 16, 32, 64, etc.

The size and width of the filters are typically odd numbers, such as 1 x 1, 3 x 3, or 5 x 5. Most common choices are 3 x 3 and 5 x 5. 

The 1 x 1 filters are used for a special purpose, which we will discuss later.


The total number of learnable parameters or weights in the first Conv layer is

- $f_k \times (f_h \times f_w \times n_k + 1)$

Note that each filter uses a bias weight.

Every single filter creates a 2D feature map $Z$. For stride $s$ and $p$ number of padding layers, the size of the 2D feature map is:

- $\frac{n_h +2p - f_h}{s} + 1 \times \frac{n_w + 2p - f_w}{s} + 1$

When SAME padding is used, the value of $p$ is determined as:

These give:
$p = \frac{f_h - 1}{2}$ or
$\frac{f_w - 1}{2}$

There will be a total of $f_k$ 2D feature maps, which are activated by passing through a nonlinear activation function.


## Pool Layer


The Pooling layer takes the activated $f_k$ 2D feature maps and downsamples to the height and width as: 

- $\frac{n_h - pool\_size}{s} + 1 \times \frac{n_w - pool\_size}{s} + 1$. 

There will be $f_k$ such pooled maps.

## FC Layer

The total number of learnable parameters or weights in an FC layer is

- $(num\_input\_neurons + 1) \times num\_output\_neurons $

One bias neuron is added with the input layer of FC.

 
## How the Layer Size Changes in Deeper Networks

Depending on the type of padding and stride, the spatial resolution decreases via convolution.
- Stride 1 & SAME padding: input map size doesn't decrease
- Stride 1 & VALID padding: input map size decreases

Depending on the type of padding and pooling widow size, the size of the feature maps decrease.
- Generally a pooling layer with stride $s$ decreases the size of the maps by a factor of $s$


Filters within a layer must all be the same size, but sizes may vary with layer. Similarly for pooling, the stride may vary with layer.

In general the number of convolutional filters increases with layers. As a consequence, as the depth of the convnet increases:
- Spatial dimension of the layers (resolution of the maps) decreases
- Depth of the layers (number of maps) increases

This results into a pyramid structure, as shown below.


<img src="https://cse.unl.edu/~hasan/Pics/CNN_Pyramid.png" width=600, height=400>


## LeNet-5

Let's put the above cheat sheet into perspective by mapping it onto the LeNet-5 architecture. We will build this convnet using TensorFlow, and use it to classify MNIST handwritten digits.

First, we stack a series of the fundamental convnet layers to create the LeNet-5 architecture.

- Conv layer C1 (with activation): 6 filters of size 5 x 5, stride 1, SAME padding, tanh activation
- Pool layer S2 (average pooling): pool size 2 x 2, stride 2
- Conv layer C3 (with activation): 16 filters of size 5 x 5, stride 1, VALID padding, tanh activation
- Pool layer S4 (max pooling): pool size 2 x 2, stride 2
- Conv layer C5 (with activation): 120 filters of size 5 x 5, stride 1, VALID padding, tanh activation
- Flatten layer
- FC layer F6: 84 neurons, tanh activation
- Softmax layer: 10 neurons, Softmax activation


A small change is made to the original LeNet-5 architecture by replacing the Gaussian activation in the final layer with the Softmax activation.


<img src="https://cse.unl.edu/~hasan/Pics/CNN_LeNet5-2.png" width=900, height=700>

In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten



# Create an empty sequential model
model = Sequential(name='LeNet-5')
 
# C1 Convolutional Layer
model.add(Conv2D(filters=6, kernel_size=(5, 5), strides=1, activation='tanh', 
                 input_shape=(28, 28, 1), padding='same'))
  
# S2 Pooling Layer
model.add(AveragePooling2D(pool_size=(2, 2), strides=2, padding='valid'))
  
# C3 Convolutional Layer
model.add(Conv2D(filters=16, kernel_size=(5, 5), strides=1, activation='tanh',
                 padding='valid'))
# S4 Pooling Layer
model.add(AveragePooling2D(pool_size=(2, 2), strides=2, padding='valid'))

# C5 Convolutional Layer
model.add(Conv2D(filters=120, kernel_size=(5, 5), strides=1,activation = 'tanh',
                  padding = 'valid'))

# Flatten the CNN output to feed it with fully connected layers
model.add(Flatten())

# FC6 Fully Connected Layer
model.add(Dense(units=84, activation='tanh'))
  
# Output layer with softmax activation
model.add(Dense(units=10, activation='softmax'))
  
# Display the model summary
model.summary()

Model: "LeNet-5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d (AveragePo (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_1 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 120)         48120     
_________________________________________________________________
flatten (Flatten)            (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 84)                1016

## Understanding the LeNet-5 Architecture

We passed single-channel (grayscale) 28 × 28 pixels images through the network. Since we used the SAME padding in the first CONV layer with a stride of 1, the images were zero-padded by adding two layers of pixels on each side. Thus the input images that were fed to the network is 32 x 32.

The rest of the network does not use any padding, which is why the size keeps shrinking as the image progresses through the network. 

The network uses average pooling. However the average pooling layers are slightly more complex than usual: each neuron computes the mean of its inputs, then multiplies the result by a learnable coefficient (one per map) and adds a learnable bias term (again, one per map), then finally applies the activation function.


Most neurons in C3 maps are connected to neurons in only three or four S2 maps (instead of all six S2 maps).

Finally, the output layer is a bit special: instead of computing the matrix multiplication of the inputs and the weight vector, each neuron outputs the square of the Euclidian distance between its input vector and its weight vector. Each output measures how much the image belongs to a particular digit class. The cross-entropy cost function is now preferred, as it penalizes bad predictions much more, producing larger gradients and converging faster.

Let's investigate the output shape of each layer as well as compute the number of learnable parameters of the network. 




#### Conv layer C1 (with activation): 6 filters of size 5 x 5, stride 1, SAME padding, tanh activation

We used $f_k = 6$ 3D filters, each with dimension $28 \times 28 \times 1$.

With stride 1 and SAME padding, the value of $p$ will be:

$p = \frac{f_h - 1}{2}$ or
$\frac{f_w - 1}{2}$

=> $p = \frac{5 - 1}{2} = 2$

Thus, the size of the output map will be:

$\frac{n_h +2p - f_h}{s} + 1 \times \frac{n_w + 2p - f_w}{s} + 1 \times f_k$

=> $\frac{28 + 2\times 2 - 5}{1} + 1 \times \frac{28 + 2\times 2 - 5}{1} + 1 \times 6$

=> $28 \times 28 \times 6$

Total number of learnable parameters or weights:

- $f_k \times (f_h \times f_w \times n_k + 1)$

=> $6 \times (5 \times 5 \times 1 + 1) = 156$

####  Pool layer S2 (average pooling): pool size 2 x 2, stride 2

The size of the average pooled map will be:

$\frac{n_h - pool\_size}{s} + 1 \times \frac{n_w - pool\_size}{s} + 1 \times f_k$

=> $\frac{28 - 2}{2} + 1 \times \frac{28 - 2}{2} + 1 \times f_k$

=> $14 \times 14 \times 6$ 

There are no learnable parameters in the pool layer.

####  Conv layer C3 (with activation): 16 filters of size 5 x 5, stride 1, VALID padding, tanh activation

With VALID padding ($p = 0$) and 16 filters, the size of the output map will be:

$\frac{n_h + 2p - f_h}{s} + 1 \times \frac{n_w + 2p - f_w}{s} + 1 \times f_k$

=> $\frac{14 + 2\times 0 - 5}{1} + 1 \times \frac{14 + 2\times 0 - 5}{1} + 1 \times 16$

=> $10 \times 10 \times 16$


Total number of learnable parameters or weights:

$f_k \times (f_h \times f_w \times n_k + 1)$
=> $16 \times (5 \times 5 \times 6 + 1) = 2416$

####  Pool layer S4 (max pooling): pool size 2 x 2, stride 2

The size of the average pooled map will be:

$\frac{n_h - pool\_size}{s} + 1 \times \frac{n_w - pool\_size}{s} + 1 \times f_k$

=> $\frac{10 - 2}{2} + 1 \times \frac{10 - 2}{2} + 1 \times 16$

=> $5 \times 5 \times 16$ 

There are no learnable parameters in the pool layer.


####  Conv layer C5 (with activation): 120 filters of size 5 x 5, stride 1, VALID padding, tanh activation


With VALID padding ($p = 0$) and 120 filters, the size of the output map will be:

$\frac{n_h + 2p - f_h}{s} + 1 \times \frac{n_w + 2p - f_w}{s} + 1 \times f_k$

=> $\frac{5 + 2\times 0 - 5}{1} + 1 \times \frac{5 + 2\times 0 - 5}{1} + 1 \times 120$

=> $1 \times 1 \times 120$

Total number of learnable parameters or weights:

$f_k \times (f_h \times f_w \times n_k + 1)$
=> $120 \times (5 \times 5 \times 16 + 1) = 48120$


#### Flatten layer

The $1 \times 1 \times 120$  input is flattened to give a vector of length $120$, which is passed as an input to the FC layer.


#### FC layer F6: 84 neurons, Sigmoid activation

Total number of learnable parameters or weights:
$(num\_input\_neurons + 1) \times num\_output\_neurons $

=> $(120 + 1) \times 84 = 10164$

#### Softmax layer: 10 neurons, Softmax activation

Total number of learnable parameters or weights:
$(num\_input\_neurons + 1) \times num\_output\_neurons $

=> $(84 + 1) \times 10 = 850$


## LeNet-5 for MNIST Classification

Now we use the LeNet-5 convnet to classify MNIST dataset.

Observe that the test accuracy is 98%.

In [2]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Compile the model
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=5, batch_size=64)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
print("\nTest Accuracy: ", test_acc)

Epoch 1/5
938/938 [==============================] - 12s 13ms/step - loss: 0.2479 - accuracy: 0.9268
Epoch 2/5
938/938 [==============================] - 12s 13ms/step - loss: 0.0944 - accuracy: 0.9704
Epoch 3/5
938/938 [==============================] - 12s 13ms/step - loss: 0.0638 - accuracy: 0.9804
Epoch 4/5
938/938 [==============================] - 12s 12ms/step - loss: 0.0484 - accuracy: 0.9847
Epoch 5/5
938/938 [==============================] - 12s 13ms/step - loss: 0.0382 - accuracy: 0.9883

Test Accuracy:  0.9866999983787537


## How Much Memory is Needed for Training Convnets?
 
 
During forward propagation of the convnet training we have to store all intermediate values (i.e., feature maps $Z$ and activated feature maps $Y$). Because these values will be required during the backward calculation.

Storing the feature maps of all layers poses a significant demand on the RAM size.

To understand this, let's consider an example.

Say that we have a mini-batch of 124 RGB images of size 224 x 224. We use 200 filters of size 5 x 5 with SAME padding and stride 1.

Thus, the size of the feature maps $Z$ will be 224 x 224 x 200.

If we use 32-bit floats to store each neuron of a feature map, then the memory required to store a single instance's feature maps will be = 224 x 224 x 200 x 32 = 321,126,400 bits = 40.14 MB

Thus, for 124 images in the mini-batch, total RAM required will be = 40.14 MB x 124 = 4977.45 MB = 4.9 GB

But this estimate is just for one Conv layer. The RAM required for the entire convnet is (at least) the total amount of RAM required by all layers.

## Memory Requirement during Prediction

However, during making prediction for a single instance we only need the RAM required by two consecutive layers. Because as soon as the next layer has been computed, the RAM occupied by the current layer can be released.


## How do we handle the memory issue during training?


We may try the following steps to avoid out-of-memory error:

- Reduce the mini-batch size.
- Reduce the size of the input images.
- Reduce dimensionality using larger strides.
- Remove a few layers.
- Use 16-bit floats.

In addition to this, distributing the computation using multiple GPUs is a useful strategy.


# What is the computational cost during training?


To answer this question, consider the following illustration. For multiple channels of the input, each channel of a single filter scans through the same region on all input channels, compute element-wise multiplication followed by sum. Note that the single multi-channel filter produces a single feature map.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_RGB_Convolution_SingleFilter.png" width=600, height=300>

Let's determine the number of floating-point multiplications required to compute a single neuron on the output feature map $Z$.

Assume that the filter size is 5 x 5 filter that scans across the 3 channels of a 224 x 224 image. Number of floating-point multiplications required for a single $Z$ neuron is 5 x 5 x 3 = 75

Total neurons on all $Z$ maps = 224 x 224 x 200 

Thus, total floating-point multiplications required to compute all $Z$ maps = 224 x 224 x 200  X 75 = 752,640,000. 

- Thus, 752 million floating-point operations just for one layer!

The formula for computing the number of floating-point operations (multiplications) or FLOPS for all filters is:

- $n_h \times n_w \times f_k \times f_h \times f_w \times n_k$

## Larger Filters vs Smaller Filters

What should be the size of the filters? Is it a good idea to use large filters?

Before we answer this question, let's consider what is the issue with large filters. 

The number of parameters (weights) for a given layer's filter is:

- $(height (f_k) \times width (f_w) \times num\_in\_channels (n_k) + 1) \times num\_out\_channels (f_k)$

Observe that the number of parameters increases significantly if both the input channels and output channels are large.

We will look at two cases.

### First Conv Layer: Typical to use large filters with stride > 1

It is not uncommon to use large filters in the first Conv layer. For example, AlexNet uses 11 x 11 filter, GoogleNet uses 7 x 7 filter, LeNet-5 uses 5 x 5 filter. Since the number of input channels is small (3 channel for RGB images) and the number of filters (output channels) is comparatively smaller, we could afford to use large filters in the first layer. 

Also typically we use larger strides with large filters in the first layer. For example, AlexNet uses stride 4, GoogleNet uses stride 2. 


### Other Conv Layers

Since the number of input and output channels increases as we go into the deeper layers in convnets, using larger filters would increase both the number of parameters and amount of floating-point multiplications. Thus, instead of using larger filters, we could stack multiple smaller filters. It has multiple benefits.

Let's illustrate this using an example. We simplify the analysis using equal number of input and output channels. Let's denote it with C.

- Single layer with a 5 x 5 filter: parameters = $5 \times  5 \times  C \times C = 25C^2$
- A stack of two layers each with 3 x 3 filters: parameters = $2(3 \times  3 \times  C \times C) = 18C^2$

Thus, we see that stacking multiple smaller layers will use fewer parameters and require fewer computations. Also, a stack of two 3 x 3 Conv (stride 1) layers has **same effective receptive field** as one 5 x 5 conv layer.

Another benefit is, if we stack $n$ smaller filters instead of a single large filter, each of those smaller filters will have their own nonlinear activation layers. It means that instead of a single activation layer now the signal will pass through three activation layers. As a consequence the decision function will be more discriminative.

Finally, relacing a larger filter with a stack of smaller filters (with incorporated noninearity) results into imposing a regularization on the larger filter. Because now we force the larger filter to have a decomposition through the smaller filters.

Thus, it is preferred to use 3 x 3 filters in the Conv layers except in the first Conv layer.

- Can we use even smaller filters, such as 1 x 1? 


## When should we use 1 x 1 filters?


Convolution using 1 x 1 filters does not seem to be useful. Because convolution is all about computing the correlation of the nearby pixels. The goal is to identify spatial features, which requires filters larger than 1 x 1. Yet so 1 x 1 filters are pretty useful. But how?

- 1 x 1 filters are used to reduce the number of output channels.

To understand this, consider the following figure. The input map has 3 channels and we convolve it with two filters (yellow and green). Each input channel of a single filter multiplies the corresponding pixel value, then add it to create a single value on the output map. The 3 x 3 input map has 9 positions. A 1 x 1 filter scans through each position and maps to 9 positions on the output map. Thus, the height and width of the input map don't change at the output. What changes is the number of output maps. Since we used two filters, the number of output maps is two. 


<img src="https://cse.unl.edu/~hasan/Pics/CNN_1_by_1_filters-1.png" width=600, height=300>

What if we have an input map with 20 channels. We can shrink the channels by convolving with 1 x 1 filters. If we need to create only 2 output maps, then we perform convolution with two 1 x 1 filters, as shown below.


<img src="https://cse.unl.edu/~hasan/Pics/CNN_1_by_1_filters-2.png" width=600, height=300>


The 1 × 1 Conv layer can be considered as constituting a fully-connected layer applied at every single pixel location to transform the corresponding $n_k$ (input channels) input values into $f_k$ (output channels) output values. The weights of the 1 x 1 Conv layer is tied across pixel location. Thus the 1 × 1 Conv layer requires $n_k \times f_k$ weights (and the bias term).

## How Do We Create an Optimal Convnet?

An optimal convnet model satisfies the following two criteria.
- Effectiveness: good performance on the test data (e.g., high test accuracy)
- Efficiency (space & time): faster training using less memmory

How do we create an optimal convnet model?

Well, it's a search problem!

There are two steps.
- Step 1: find an optimal architecture.
- Step 2: find optimal hyperparameters.


### Step 1: Find an Optimal Architecture

A convnet is built by stacking the following layers Conv, Pool, FC, etc. We can combine these layers in many ways. Thus, the convnet architecture is super flexible. Also, resource permitting, we may build a very wide and very deep convnet. We shouldn't necessarily be constrained within sequential flow. We may add parallel connections.

How do we decide which architecture would be suitable for our problem? Another question is, how do we find the optimal hyperparameters?

### Step 2: Find Optimal Hyperparameters

A straight-forward idea is to perform hyperparameter tuning via cross-validation. But that would be very expensive.


It appears that finding the optimal architecture as well as optimal hyperparameters is a tricky business. It's a "black art". We need mathematical insights and time plus resource to perform a lot of trial and error. We should also be able to understand how to apply engineerng tricks. But above all we need to develop scientific intuition to guide the trial and error process.

- What is the best way to develop this intuition?

Well, we should start with meditating on the sucessful convnet architectures. This study will reveal what worked for other people, and more importantly, what trick (engineering or "black art") we could apply to advance the state of the art of convnets architectures. 

Let's start this medidatation ...